### We are going to use leftover food at home and use Personal Chef to search online and get receipe suggestion.


In [1]:
!pip install -U langchain openai langchain-openai langchain_community tavily-python langsmith  wikipedia

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.2/111.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.2/319.2 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.3/496.3 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 5.0 MB/s eta 0:00:00
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=2710f9f0921ed599a4e1d7a5caa57272ecabea20f8f7cee4fc1feb5c8cad570d
  Stored in directory: /root/.cache/pip/wheels/63/47/7c/a9688349aa74d228ce0a9023229c6c0ac52ca2a4

In [2]:
!pip install tavily-python

In [5]:
from google.colab import userdata
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')

In [6]:
from google.colab import userdata
TAVILY_API_KEY=userdata.get('tavily_key')

In [15]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.langsmith.com"
os.environ["LANGCHAIN_PROJECT"] = "PersonalChef"

In [16]:
from langchain.tools import tool
from typing import Dict, Any
from tavily import TavilyClient

# Search for Receipe, LLM will search for latest receipe and make use of those
tavily_client = TavilyClient(TAVILY_API_KEY)

@tool
def web_search(query: str) -> Dict[str, Any]:
    """Search the web for information"""
    return tavily_client.search(query)

In [17]:
# Agent Role and Work
system_prompt = """

You are a personal chef. The user will give you a list of ingredients they have left over in their house.

Using the web search tool, search the web for recipes that can be made with the ingredients they have.

Return recipe suggestions and eventually the recipe instructions to the user, if requested.

"""

In [18]:
from langchain.chat_models import init_chat_model

model = init_chat_model(model="gpt-4o-mini",
                        model_provider="openai",
                        api_key=OPENAI_API_KEY)

In [19]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
agent = create_agent(
    model=model,
    tools=[web_search],
    system_prompt=system_prompt,
    checkpointer=InMemorySaver()
)

In [20]:
from langchain.messages import HumanMessage
config = {"configurable": {"thread_id": "1"}}
response = agent.invoke(
    {"messages": [HumanMessage(content="I have some leftover rajma and rice. What can I make?")]},
    config
)
print(response['messages'][-1].content)

Here are some recipe suggestions for using your leftover rajma and rice:

1. **Rajma Chawal (Rajma Masala)**
   - [Rajma Recipe | Rajma Masala | Rajma Chawal](https://www.indianhealthyrecipes.com/rajma-recipe-rajma-masala-recipe/)
   - A delicious curried dish of red kidney beans served with rice.

2. **Rajma Masala**
   - [Rajma Chawal (Rajma Masala)](https://upbeetanisha.com/rajma-masala/)
   - A hearty and flavorful kidney bean dish perfect for any time of the year.

3. **Rajma Pulao**
   - [Rajma Pulao Recipe](https://rakskitchen.net/rajma-pulao-recipe/)
   - A rice dish made with rajma and flavorful ingredients.

4. **Delhi Style Rajma Chawal**
   - [Rajma Chawal Delhi Style](https://indishious.com/en/recipes/rajma-chawal/)
   - A traditional way to serve rajma with rice.

5. **Healthy Rajma with Basmati Rice**
   - [Rajma (Healthy Indian Kidney Bean Stew) with Basmati Rice](https://dadaeats.substack.com/p/rajma-healthy-indian-kidney-bean)
   - A plant-based protein dish, this rec